# Push Cart Rules Handler

This module receives cart rule changes from Module 2 or Module 3 and pushes them to the MaxAB API.

## Workflow
1. Receive cart rule data from Module 2 (daily reset) or Module 3 (periodic adjustments)
2. Filter to only SKUs where cart rule actually changed
3. Aggregate cart rules by cohort using stock-weighted average
4. Expand to all packing units with adjusted cart rules:
   - Min PU (basic unit): Uses the original cart rule
   - Bigger PUs: cart_rule / basic_unit_count, rounded, then +10%
5. Split into chunks and upload via MaxAB API

## Required Input Columns
- `product_id`: Product ID
- `sku`: SKU name
- `new_cart_rule`: New cart rule to set (for basic/min packing unit)
- `warehouse_id`: Warehouse ID
- `cohort_id`: Cohort ID for pricing
- `stocks`: Current stock level (used for weighted aggregation)
- `current_cart_rule`: Current cart rule (for filtering changed rules)

## Output
- Uploads cart rules sheets to MaxAB API per cohort
- Returns summary dict with counts of pushed/failed/skipped


In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================
import pandas as pd
import numpy as np
import json
import time
import base64
import os
from datetime import datetime
import pytz

# AWS for secrets management
import boto3
from botocore.exceptions import ClientError

# HTTP requests for API calls
import requests

# Progress bar for chunk uploads
from tqdm import tqdm

# Import setup_environment_2 for credentials
import sys
sys.path.append('..')
import setup_environment_2

# Cairo timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)

# =============================================================================
# CONFIGURATION
# =============================================================================
CHUNK_SIZE_DEFAULT = 4000   # Default chunk size for API uploads
CHUNK_SIZE_SPECIAL = 2000   # Smaller chunk size for specific cohorts (e.g., cohort 61)
UPLOAD_DIR = 'uploads_cart_rules'      # Directory to save full upload files
MANUAL_DIR = 'manual_cart_rules'       # Directory to save chunk files

# Cart rule adjustment for bigger packing units
BIGGER_PU_MULTIPLIER = 1.10  # 10% increase for bigger packing units

print(f"Push Cart Rules Handler loaded at {CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S')} Cairo time")


In [ ]:
# =============================================================================
# AWS & API FUNCTIONS
# =============================================================================

def get_secret(secret_name: str) -> str:
    """
    Retrieve a secret from AWS Secrets Manager.
    
    This function connects to AWS Secrets Manager to fetch sensitive credentials
    (like API keys, passwords) securely without hardcoding them in the code.
    
    Args:
        secret_name: The name/path of the secret in AWS Secrets Manager
                    (e.g., "prod/pricing/api/")
    
    Returns:
        Secret string (JSON format) or decoded binary
        
    Raises:
        ClientError: If the secret cannot be retrieved (permissions, not found, etc.)
    """
    region_name = "us-east-1"
    session = boto3.session.Session()
    client = session.client(service_name='secretsmanager', region_name=region_name)

    try:
        response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        error_code = e.response['Error']['Code']
        error_messages = {
            'DecryptionFailureException': "Can't decrypt secret using provided KMS key",
            'InternalServiceErrorException': "Server-side error occurred",
            'InvalidParameterException': "Invalid parameter value provided",
            'InvalidRequestException': "Invalid request for current resource state",
            'ResourceNotFoundException': "Requested resource not found"
        }
        if error_code in error_messages:
            print(f"AWS Error: {error_messages[error_code]}")
        raise e
    
    if 'SecretString' in response:
        return response['SecretString']
    return base64.b64decode(response['SecretBinary'])


def get_access_token(url: str, client_id: str, client_secret: str) -> str:
    """
    Get OAuth2 access token for MaxAB API authentication.
    
    Uses password grant type to exchange credentials for an access token.
    The token is used in subsequent API calls for authorization.
    
    Args:
        url: The OAuth token endpoint URL
        client_id: OAuth client ID for the application
        client_secret: OAuth client secret for the application
    
    Returns:
        Access token string to be used in Authorization header
    """
    response = requests.post(
        url,
        data={
            "grant_type": "password",
            "username": API_USERNAME,
            "password": API_PASSWORD
        },
        auth=(client_id, client_secret),
    )
    return response.json()["access_token"]


def _get_api_token() -> str:
    """
    Get a fresh API token for MaxAB API requests.
    
    This is a convenience wrapper that calls get_access_token with
    the correct MaxAB SSO endpoint and client credentials.
    
    Returns:
        Fresh access token string
    """
    return get_access_token(
        'https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
        'main-system-externals',
        API_SECRET
    )


def post_cart_rules(cohort_id: int, file_name: str) -> requests.Response:
    """
    Upload a cart rules Excel sheet to MaxAB API for a specific cohort.
    
    This function:
    1. Gets a fresh API token
    2. Prepares the file for multipart upload
    3. POSTs to the cohort cart-rules endpoint
    
    Args:
        cohort_id: The target cohort ID to upload cart rules for
        file_name: Path to the Excel file containing cart rules data
                  Expected columns: Product ID, Packing Unit ID, Cart Rules
    
    Returns:
        requests.Response object with API response
        Check response.content for 'Cart Rules Updated Successfully!' to verify upload
    """
    token = _get_api_token()
    url = f"https://api.maxab.info/main-system/api/admin-portal/cohorts/{cohort_id}/cart-rules"
    
    files = [('sheet', (file_name, open(file_name, 'rb'), 
              'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))]
    headers = {'Authorization': f'bearer {token}'}
    
    return requests.post(url, headers=headers, data={}, files=files)


In [ ]:
# =============================================================================
# API CREDENTIALS INITIALIZATION
# =============================================================================
# Load API credentials from AWS Secrets Manager
# These are used by get_access_token() to authenticate with MaxAB API

pricing_api_secret = json.loads(get_secret("prod/pricing/api/"))
API_USERNAME = pricing_api_secret["egypt_username"]
API_PASSWORD = pricing_api_secret["egypt_password"]
API_SECRET = pricing_api_secret["egypt_secret"]

print("✓ API credentials loaded successfully")


In [ ]:
# =============================================================================
# DATA PREPARATION FUNCTIONS
# =============================================================================

def cart_rules_preparation(df_cart_rules: pd.DataFrame, pus: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare cart rule data for API upload by aggregating and expanding to packing units.
    
    This function performs the following transformation:
    1. Since each cohort may have multiple warehouses, we take the MINIMUM
       cart rule per product-cohort combination (most restrictive)
    2. Then we expand to all packing units with DIFFERENT cart rules:
       - Min/basic PU: Uses the original cart rule
       - Bigger PUs: cart_rule / basic_unit_count, rounded, then +10%
    
    Cart Rule Calculation for Bigger Packing Units:
    ------------------------------------------------
    The cart rule from the main script is for the minimum packing unit (e.g., single bottle).
    For bigger packing units (e.g., pack of 6):
    - Divide the cart rule by basic_unit_count
    - Round to nearest integer
    - Increase by 10% (to give slightly more flexibility for bulk purchases)
    
    Example:
    - Min PU cart rule = 30 (single bottles)
    - Pack of 6: 30 / 6 = 5, then 5 * 1.10 = 5.5, rounded = 6 packs
    
    Minimum Cart Rule Across Warehouses:
    ------------------------------------
    For each product in a cohort, we take the MINIMUM cart rule across all warehouses.
    This ensures the most restrictive limit applies.
    
    Example:
    - Product 1, Warehouse A: cart_rule = 10
    - Product 1, Warehouse B: cart_rule = 5
    - Final cart rule = 5 (the minimum)
    
    Args:
        df_cart_rules: DataFrame with columns:
            - product_id: Product identifier
            - sku: SKU name  
            - cohort_id: Target cohort
            - warehouse_id: Source warehouse
            - stocks: Stock quantity (not used for cart rule aggregation)
            - new_cart_rule: New cart rule for basic/min packing unit
            
        pus: Packing units DataFrame with columns:
            - product_id: Product identifier
            - packing_unit_id: Packing unit ID
            - basic_unit_count: Number of basic units in this packing unit
    
    Returns:
        DataFrame with columns:
            - product_id, sku, cohort_id: Identifiers
            - final_cart_rule: Minimum cart rule across warehouses (for min PU)
            - packing_unit_id: Packing unit ID
            - basic_unit_count: Units per pack
            - final_pu_cart_rule: Adjusted cart rule for this packing unit
    """
    # Step 1: Aggregate to get MINIMUM cart rule per product-cohort
    # Take the minimum cart rule across all warehouses (most restrictive)
    df_final = df_cart_rules.groupby(['product_id', 'sku', 'cohort_id']).agg({
        'new_cart_rule': 'min'  # Take minimum across warehouses
    }).reset_index()
    
    # Rename column for clarity
    df_final = df_final.rename(columns={'new_cart_rule': 'final_cart_rule'})
    
    # Step 2: Ensure cart rule is integer
    df_final['final_cart_rule'] = df_final['final_cart_rule'].round().astype(int)
    
    # Step 3: Expand to all packing units
    df_final = df_final.merge(pus, on=['product_id'])
    
    # Step 4: Calculate packing unit cart rule
    # For min PU (basic_unit_count = 1): use the original cart rule
    # For bigger PUs: divide by basic_unit_count, multiply by 1.10, THEN round
    df_final['final_pu_cart_rule'] = df_final.apply(
        lambda row: row['final_cart_rule'] if row['basic_unit_count'] == 1 
        else int(round((row['final_cart_rule'] / row['basic_unit_count']) * BIGGER_PU_MULTIPLIER)),
        axis=1
    )
    
    # Ensure cart rules are within valid range: min 2, max 150
    df_final['final_pu_cart_rule'] = df_final['final_pu_cart_rule'].clip(lower=2, upper=150)
    
    return df_final


In [ ]:
# =============================================================================
# MAIN PUSH CART RULES FUNCTION
# =============================================================================

# Mode constants for clarity
MODE_TESTING = 'testing'  # Prepare files but DON'T upload to API
MODE_LIVE = 'live'        # Prepare files AND upload to API

def push_cart_rules(df_cart_rules: pd.DataFrame, pus: pd.DataFrame, 
                    source_module: str = 'unknown',
                    mode: str = 'testing') -> dict:
    """
    Main entry point: Push cart rule changes to MaxAB API.
    
    This function orchestrates the entire cart rules upload workflow:
    1. Filter to only changed cart rules
    2. Prepare data (stock-weighted aggregation, packing unit expansion)
    3. Apply cart rule adjustments for bigger packing units
    4. Split into chunks and upload via API (only if mode='live')
    
    ⚠️ IMPORTANT: Default mode is 'testing' - cart rules will NOT be pushed!
    Set mode='live' to actually upload cart rules to the API.
    
    Cart Rule Logic for Packing Units:
    -----------------------------------
    - Min/basic PU (basic_unit_count=1): Uses original cart rule
    - Bigger PUs: cart_rule / basic_unit_count, rounded, then +10%
    
    Example:
    - Product has cart rule = 30 (for single bottles)
    - Single bottle (basic_unit_count=1): cart = 30
    - Pack of 6 (basic_unit_count=6): cart = round(30/6) * 1.10 = 6
    
    Parameters:
    -----------
    df_cart_rules : pd.DataFrame
        DataFrame with cart rule recommendations from Module 2 or 3
        Required columns: product_id, sku, new_cart_rule, warehouse_id, cohort_id, 
                         stocks, current_cart_rule
        
    pus : pd.DataFrame
        Packing units lookup table
        Required columns: product_id, packing_unit_id, basic_unit_count
        
    source_module : str
        Identifier for calling module ('module_2' or 'module_3')
        Used in logging and file naming
        
    mode : str
        'testing' (default) - Prepare and save files, but DON'T upload to API
        'live' - Prepare files AND upload to MaxAB API
        
    Returns:
    --------
    dict with results:
        - total_received: Input row count
        - cart_rule_changes: Rows with actual cart rule changes
        - skipped: Rows with no change
        - pushed: Successfully uploaded (or would be in testing mode)
        - failed: Upload failures
        - source_module: Which module called this
        - timestamp: When upload completed
        - mode: 'testing' or 'live'
    """
    
    # Validate mode parameter
    if mode not in [MODE_TESTING, MODE_LIVE]:
        print(f"⚠️ Invalid mode '{mode}'. Using 'testing' mode.")
        mode = MODE_TESTING
    
    # Print clear mode indicator
    print(f"\n{'🧪' if mode == MODE_TESTING else '🚀'} MODE: {mode.upper()}")
    if mode == MODE_TESTING:
        print("   Files will be prepared but NOT uploaded to API")
    else:
        print("   Files will be prepared AND uploaded to API")
    
    # Initialize result tracking
    result = {
        'total_received': len(df_cart_rules),
        'cart_rule_changes': 0,
        'pushed': 0,
        'failed': 0,
        'skipped': 0,
        'source_module': source_module,
        'timestamp': datetime.now(CAIRO_TZ).strftime('%Y-%m-%d %H:%M:%S'),
        'mode': mode,
        'failed_cohorts': []  # List of cohorts that failed to upload
    }
    
    # =========================================================================
    # STEP 1: Validate input
    # =========================================================================
    if df_cart_rules.empty:
        print(f"⚠️ No data received from {source_module}")
        return result
    
    # Filter to only rows where cart rule actually changed
    df_to_push = df_cart_rules[
        (df_cart_rules['new_cart_rule'].notna()) & 
        (df_cart_rules['new_cart_rule'] != df_cart_rules['current_cart_rule'])
    ].copy()
    
    result['cart_rule_changes'] = len(df_to_push)
    result['skipped'] = len(df_cart_rules) - len(df_to_push)
    
    if df_to_push.empty:
        print(f"ℹ️ No cart rule changes to push from {source_module}")
        return result
    
    print(f"\n{'='*60}")
    print(f"PUSH CART RULES - Source: {source_module}")
    print(f"{'='*60}")
    print(f"Total received: {result['total_received']}")
    print(f"Cart rule changes to push: {result['cart_rule_changes']}")
    print(f"Skipped (no change): {result['skipped']}")
    
    # Check required columns
    required_cols = ['product_id', 'sku', 'new_cart_rule', 'warehouse_id', 'cohort_id', 'stocks']
    missing_cols = [c for c in required_cols if c not in df_to_push.columns]
    if missing_cols:
        print(f"❌ ERROR: Missing required columns: {missing_cols}")
        result['failed'] = len(df_to_push)
        return result
    
    # =========================================================================
    # STEP 2: Log cart rule change summary
    # =========================================================================
    print(f"\nCart rule changes summary:")
    if 'current_cart_rule' in df_to_push.columns:
        increases = len(df_to_push[df_to_push['new_cart_rule'] > df_to_push['current_cart_rule']])
        decreases = len(df_to_push[df_to_push['new_cart_rule'] < df_to_push['current_cart_rule']])
        print(f"  Increases: {increases}")
        print(f"  Decreases: {decreases}")
    
    # =========================================================================
    # STEP 3: Prepare data (aggregate by cohort, expand to packing units)
    # =========================================================================
    push_data = df_to_push[required_cols].copy()
    final_data = cart_rules_preparation(push_data, pus)
    
    # Store for debugging/inspection
    global cart_rules_to_push
    cart_rules_to_push = final_data
    
    print(f"\n📋 Prepared {len(final_data)} packing unit cart rules")
    
    # Show sample of cart rule adjustments for different packing units
    sample_products = final_data.groupby('product_id').filter(lambda x: len(x) > 1).head(10)
    if not sample_products.empty:
        print(f"\nSample cart rule adjustments (showing products with multiple PUs):")
        print(sample_products[['product_id', 'basic_unit_count', 'final_cart_rule', 'final_pu_cart_rule']].to_string(index=False))
    
    # =========================================================================
    # STEP 4: Upload to API per cohort
    # =========================================================================
    # Ensure output directories exist
    os.makedirs(UPLOAD_DIR, exist_ok=True)
    os.makedirs(MANUAL_DIR, exist_ok=True)
    
    total_pushed = 0
    total_failed = 0
    
    for cohort in final_data.cohort_id.unique():
        print(f"\n{'='*50}")
        print(f"Processing cohort: {cohort}")
        print('='*50)
        
        # Filter to this cohort
        cohort_data = final_data[final_data['cohort_id'] == cohort].copy()
        
        # Prepare output DataFrame with API-expected columns
        # Cart rules API expects: Product ID, Packing Unit ID, Cart Rules
        out = cohort_data[['product_id', 'packing_unit_id', 'final_pu_cart_rule']].copy()
        out.columns = ['Product ID', 'Packing Unit ID', 'Cart Rules']
        
        # Drop duplicates
        out = out.drop_duplicates()
        
        # Filter out invalid cart rules (min 2, max 150)
        out = out[(out['Cart Rules'] >= 2) & (out['Cart Rules'] <= 150)].reset_index(drop=True)
        
        if len(out) == 0:
            print(f"  No valid cart rules for cohort {cohort}")
            continue
        
        # Save full file for reference
        file_name_ = f'{UPLOAD_DIR}/{source_module}_cart_rules_{cohort}.xlsx'
        out.to_excel(file_name_, index=False, engine='xlsxwriter')
        print(f"  Saved: {file_name_} ({len(out)} rows)")
        time.sleep(2)
        
        # In testing mode, skip the actual API upload
        if mode == MODE_TESTING:
            print(f"  🧪 [TESTING] Would upload {len(out)} cart rules (skipped)")
            total_pushed += len(out)
            continue
        
        # Split into chunks for API upload
        chunk_size = CHUNK_SIZE_SPECIAL if cohort == 61 else CHUNK_SIZE_DEFAULT
        chunks = [out[i:i + chunk_size] for i in range(0, len(out), chunk_size)]
        print(f"  Split into {len(chunks)} chunks (size: {chunk_size})")
        
        # Save and upload chunks
        fileslist = []
        for i, chunk in tqdm(enumerate(chunks), total=len(chunks), desc="  Saving chunks"):
            output_file = f'{MANUAL_DIR}/cart_rules_{cohort}_chunk_{i + 1}.xlsx'
            fileslist.append(output_file)
            chunk.to_excel(output_file, index=False, engine='xlsxwriter')
        
        # Upload each chunk
        print("  Uploading...")
        upload_success = True
        
        for file in fileslist:
            chunk_num = file.split('chunk_')[1].split('.xls')[0]
            response = post_cart_rules(cohort, file)
            
            if 'Cart Rules Updated Successfully!' in str(response.content):
                print(f"    ✓ Chunk {chunk_num} uploaded successfully")
                total_pushed += len(pd.read_excel(file))
            else:
                print(f"    ✗ ERROR chunk {chunk_num}")
                print(f"      Response: {response.content}")
                upload_success = False
                total_failed += len(pd.read_excel(file))
                break
        
        if not upload_success:
            print(f"  Upload failed for cohort {cohort}")
            result['failed_cohorts'].append(cohort)
    
    # =========================================================================
    # STEP 5: Final summary
    # =========================================================================
    result['pushed'] = total_pushed
    result['failed'] = total_failed
    
    print(f"\n{'='*60}")
    if mode == MODE_TESTING:
        print("🧪 TESTING MODE COMPLETE - NO CART RULES WERE UPLOADED")
    else:
        print("🚀 UPLOAD COMPLETE")
    print(f"{'='*60}")
    print(f"Mode: {mode}")
    print(f"Total prepared: {total_pushed}")
    if mode == MODE_LIVE:
        print(f"Total failed: {total_failed}")
        if result['failed_cohorts']:
            print(f"Failed cohorts: {result['failed_cohorts']}")
    
    return result


# Global variable to store prepared cart rules for debugging/inspection
cart_rules_to_push = pd.DataFrame()
